In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin



---





---



# **Detección de balón arco (resultado a color)**

Este código procesa un video, detectando objetos con YOLO mientras mantiene el video a color, aunque la detección se haga en escala de grises.

**📌 En resumen:**


*   Carga un video de entrada.
*   Convierte cada fotograma a escala de grises para hacer la detección.
*   Redimensiona la imagen a 640x640 (para ajustarse al modelo YOLO).
*   Convierte la imagen de grises a BGR antes de pasarla a YOLO.
*   Usa YOLO para detectar objetos y obtener coordenadas de las cajas.
*   Dibuja las cajas sobre el fotograma original en color (para que el video de salida se mantenga en color).
*   Guarda el video con las detecciones en color.


In [ ]:
pip install --upgrade torch ultralytics

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Intentamos configurar torch._dynamo para saltar los hooks de FSDP
try:
    import torch._dynamo
    torch._dynamo.config.skip_fsdp_hooks = True
except AttributeError:
    print("No se pudo configurar torch._dynamo.config. Revisa la versión de PyTorch.")

class VideoProcessor:
    def __init__(self, model_path):
        # Carga del modelo personalizado
        self.model = YOLO(model_path)
        # Mapeo de clases: ajusta según corresponda a tu entrenamiento
        self.class_names = {0: "balon", 1: "arco"}
        # Mapeo de colores en formato BGR
        self.colors = {
            "balón": (0, 0, 255),      # rojo
            "arco": (255, 0, 0)   # azul
        }

    def process_video(self, input_path, output_path):
        # Abrimos el video de entrada
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError(f"No se pudo abrir el video: {input_path}")

        # Obtenemos las propiedades del video original
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        # Procesamos cada fotograma del video
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Guardamos una copia del frame original a color
            original_frame = frame.copy()

            # Convertir a escala de grises para detección
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (640, 640))

            # Convertimos de nuevo a 3 canales para YOLO (BGR)
            input_frame = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)

            # Hacer la detección en la imagen en escala de grises con 3 canales
            results = self.model(input_frame)

            # Extraemos las cajas detectadas, sus coordenadas y clases
            boxes = results[0].boxes
            xyxy = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            classes = boxes.cls.cpu().numpy()

            # Calculamos el factor de escala para volver a mapear las coordenadas al tamaño original del video
            scale_x = width / 640
            scale_y = height / 640

            # Dibujamos las cajas de detección en el fotograma original a color
            for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, classes):
                # Convertimos las coordenadas de vuelta al tamaño original
                x1 = int(x1 * scale_x)
                y1 = int(y1 * scale_y)
                x2 = int(x2 * scale_x)
                y2 = int(y2 * scale_y)

                label = self.class_names.get(int(cls), str(cls))
                color = self.colors.get(label, (0, 255, 0))
                cv2.rectangle(original_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(original_frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Escribir el fotograma procesado con detecciones en el video de salida
            out.write(original_frame)

        cap.release()
        out.release()

        return output_path



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
processor = VideoProcessor("/content/best.pt")
video_procesado = processor.process_video(
    input_path=r"/content/Sample8.mp4",
    output_path=r"/content/SampleFT8.mp4"
)



0: 640x640 1 ball, 1 goal_post, 536.2ms
Speed: 4.5ms preprocess, 536.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 478.9ms
Speed: 3.6ms preprocess, 478.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 496.3ms
Speed: 3.9ms preprocess, 496.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 490.5ms
Speed: 3.2ms preprocess, 490.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 498.3ms
Speed: 3.2ms preprocess, 498.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 479.5ms
Speed: 3.3ms preprocess, 479.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_post, 502.9ms
Speed: 3.2ms preprocess, 502.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ball, 1 goal_



---



# **Tracking balón**

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

class VideoProcessor:
    def __init__(self, model_path):
        # Carga del modelo personalizado
        self.model = YOLO(model_path)
        # Mapeo de clases: ajusta según corresponda a tu entrenamiento
        self.class_names = {0: "balon", 1: "arco"}
        # Mapeo de colores en formato BGR
        self.colors = {
            "balon": (0, 0, 255),      # rojo
            "arco": (255, 0, 0)        # azul
        }
        # Tracker CSRT de OpenCV (inicialmente None)
        self.tracker = None
        self.last_ball_bbox = None  # [x, y, w, h] en coordenadas de frame original

    def process_video(self, input_path, output_path):
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError(f"No se pudo abrir el video: {input_path}")

        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            original_frame = frame.copy()

            # Preprocesamiento para YOLO
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (640, 640))
            input_frame = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)

            # Detección con YOLO
            results = self.model(input_frame)
            boxes = results[0].boxes
            xyxy = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            classes = boxes.cls.cpu().numpy()

            # Escala de coordenadas
            scale_x = width / 640
            scale_y = height / 640

            # Filtramos solo la pelota (clase 0) y tomamos la de mayor confianza
            ball_indices = [i for i, cls in enumerate(classes) if int(cls) == 0]
            if ball_indices:
                # Hay al menos una detección de pelota
                # Escogemos la de mayor confianza
                best_i = max(ball_indices, key=lambda i: confs[i])
                x1, y1, x2, y2 = xyxy[best_i]
                # Convertir a coordenadas originales
                x1, y1 = int(x1 * scale_x), int(y1 * scale_y)
                x2, y2 = int(x2 * scale_x), int(y2 * scale_y)
                w, h = x2 - x1, y2 - y1

                # Dibujar YOLO
                cv2.rectangle(original_frame, (x1, y1), (x2, y2), self.colors["balon"], 2)
                cv2.putText(original_frame, f"balon {confs[best_i]:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)

                # (Re)inicializar tracker CSRT con la caja detectada
                self.tracker = cv2.TrackerCSRT_create()
                self.last_ball_bbox = (x1, y1, w, h)
                self.tracker.init(original_frame, self.last_ball_bbox)

            else:
                # YOLO no detectó la pelota: usar tracker como respaldo
                if self.tracker is not None:
                    ok, bbox = self.tracker.update(original_frame)
                    if ok:
                        # Tracker encontró la pelota
                        x, y, w, h = [int(v) for v in bbox]
                        cv2.rectangle(original_frame, (x, y), (x + w, y + h), self.colors["balon"], 2)
                        cv2.putText(original_frame, "balon (track)", (x, y - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)
                        self.last_ball_bbox = (x, y, w, h)
                    else:
                        # Tracker falló: descartarlo hasta próxima detección
                        self.tracker = None
                        self.last_ball_bbox = None

            # Además dibujamos detecciones de "arco" si las hay
            for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, classes):
                if int(cls) == 1:  # clase "arco"
                    x1, y1 = int(x1 * scale_x), int(y1 * scale_y)
                    x2, y2 = int(x2 * scale_x), int(y2 * scale_y)
                    cv2.rectangle(original_frame, (x1, y1), (x2, y2), self.colors["arco"], 2)
                    cv2.putText(original_frame, f"arco {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["arco"], 2)

            out.write(original_frame)

        cap.release()
        out.release()
        return output_path


In [ ]:
processor = VideoProcessor("/content/best.pt")
video_procesado = processor.process_video(
    input_path=r"/content/Sample5.mp4",
    output_path=r"/content/Sample5seguimiente.mp4"
)



---



# **Cuadricula Cancha + detección puntaje**

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

class VideoProcessor:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.class_names = {0: "balon", 1: "arco"}
        self.colors = {
            "balon": (0, 0, 255),  # rojo
            "arco": (255, 0, 0)    # azul
        }
        self.tracker = None
        self.last_ball_bbox = None
        self.puntaje_asignado = None  # <- nuevo

    def process_video(self, input_path, output_path):
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError(f"No se pudo abrir el video: {input_path}")

        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            original_frame = frame.copy()

            # Preprocesamiento para YOLO
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (640, 640))
            input_frame = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)

            results = self.model(input_frame)
            boxes = results[0].boxes
            xyxy = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            classes = boxes.cls.cpu().numpy()

            scale_x = width / 640
            scale_y = height / 640

            ball_indices = [i for i, cls in enumerate(classes) if int(cls) == 0]
            if ball_indices:
                best_i = max(ball_indices, key=lambda i: confs[i])
                x1, y1, x2, y2 = xyxy[best_i]
                x1, y1 = int(x1 * scale_x), int(y1 * scale_y)
                x2, y2 = int(x2 * scale_x), int(y2 * scale_y)
                w, h = x2 - x1, y2 - y1

                cv2.rectangle(original_frame, (x1, y1), (x2, y2), self.colors["balon"], 2)
                cv2.putText(original_frame, f"balon {confs[best_i]:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)

                self.tracker = cv2.TrackerCSRT_create()
                self.last_ball_bbox = (x1, y1, w, h)
                self.tracker.init(original_frame, self.last_ball_bbox)
            else:
                if self.tracker is not None:
                    ok, bbox = self.tracker.update(original_frame)
                    if ok:
                        x, y, w, h = [int(v) for v in bbox]
                        cv2.rectangle(original_frame, (x, y), (x + w, y + h), self.colors["balon"], 2)
                        cv2.putText(original_frame, "balon (track)", (x, y - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)
                        self.last_ball_bbox = (x, y, w, h)
                    else:
                        self.tracker = None
                        self.last_ball_bbox = None

            for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, classes):
                if int(cls) == 1:  # clase "arco"
                    x1, y1 = int(x1 * scale_x), int(y1 * scale_y)
                    x2, y2 = int(x2 * scale_x), int(y2 * scale_y)
                    cv2.rectangle(original_frame, (x1, y1), (x2, y2), self.colors["arco"], 2)
                    cv2.putText(original_frame, f"arco {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["arco"], 2)

                    # Dibujar cuadrícula
                    cell_w = (x2 - x1) // 3
                    cell_h = (y2 - y1) // 3
                    puntajes = [[5, 2, 5],
                                [3, 1, 3],
                                [5, 1, 5]]

                    for i in range(3):
                        for j in range(3):
                            cx1 = x1 + j * cell_w
                            cy1 = y1 + i * cell_h
                            cx2 = cx1 + cell_w
                            cy2 = cy1 + cell_h
                            cv2.rectangle(original_frame, (cx1, cy1), (cx2, cy2), (0, 255, 0), 1)
                            cv2.putText(original_frame, str(puntajes[i][j]), (cx1 + 5, cy1 + 20),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                    # Determinar puntaje si entra el balón
                    if self.last_ball_bbox is not None and self.puntaje_asignado is None:
                        bx, by, bw, bh = self.last_ball_bbox
                        ball_cx = bx + bw // 2
                        ball_cy = by + bh // 2

                        if x1 <= ball_cx <= x2 and y1 <= ball_cy <= y2:
                            col = (ball_cx - x1) // cell_w
                            row = (ball_cy - y1) // cell_h
                            col = int(min(max(col, 0), 2))
                            row = int(min(max(row, 0), 2))
                            self.puntaje_asignado = puntajes[row][col]

                            cv2.putText(original_frame, f"PUNTAJE: {self.puntaje_asignado}", (50, 50),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

            out.write(original_frame)

        cap.release()
        out.release()

        if self.puntaje_asignado is not None:
            print(f"Puntaje asignado: {self.puntaje_asignado}")
        else:
            print("No se detectó un gol en el video.")

        return output_path


In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

class VideoProcessor:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.class_names = {0: "balon", 1: "arco"}
        self.colors = {
            "balon": (0, 0, 255),  # rojo
            "arco": (255, 0, 0)    # azul
        }

    def process_video(self, input_path, output_path):
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError(f"No se pudo abrir el video: {input_path}")

        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        tracker = None
        last_ball_bbox = None
        last_arco_bbox = None

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            original_frame = frame.copy()

            # Preprocesamiento para YOLO
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (640, 640))
            input_frame = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)

            results = self.model(input_frame)
            boxes = results[0].boxes
            xyxy = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            classes = boxes.cls.cpu().numpy()

            scale_x = width / 640
            scale_y = height / 640

            # 1) Detectar balón (mejor confianza)
            ball_indices = [i for i, cls in enumerate(classes) if int(cls) == 0]
            if ball_indices:
                best_i = max(ball_indices, key=lambda i: confs[i])
                x1, y1, x2, y2 = xyxy[best_i]
                x1, y1 = int(x1 * scale_x), int(y1 * scale_y)
                x2, y2 = int(x2 * scale_x), int(y2 * scale_y)
                w, h = x2 - x1, y2 - y1

                cv2.rectangle(original_frame, (x1, y1), (x2, y2), self.colors["balon"], 2)
                cv2.putText(original_frame, f"balon {confs[best_i]:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)

                # reiniciar tracker
                tracker = cv2.TrackerCSRT_create()
                tracker.init(original_frame, (x1, y1, w, h))
                last_ball_bbox = (x1, y1, w, h)
            else:
                # tracking si no detecta
                if tracker is not None:
                    ok, bbox = tracker.update(original_frame)
                    if ok:
                        x, y, w, h = [int(v) for v in bbox]
                        cv2.rectangle(original_frame, (x, y), (x + w, y + h), self.colors["balon"], 2)
                        cv2.putText(original_frame, "balon (track)", (x, y - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["balon"], 2)
                        last_ball_bbox = (x, y, w, h)
                    else:
                        tracker = None

            # 2) Detectar arco (podría haber múltiples, tomamos el de mayor confianza)
            arco_indices = [i for i, cls in enumerate(classes) if int(cls) == 1]
            if arco_indices:
                best_j = max(arco_indices, key=lambda i: confs[i])
                ax1, ay1, ax2, ay2 = xyxy[best_j]
                ax1, ay1 = int(ax1 * scale_x), int(ay1 * scale_y)
                ax2, ay2 = int(ax2 * scale_x), int(ay2 * scale_y)

                cv2.rectangle(original_frame, (ax1, ay1), (ax2, ay2), self.colors["arco"], 2)
                cv2.putText(original_frame, f"arco {confs[best_j]:.2f}", (ax1, ay1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors["arco"], 2)

                last_arco_bbox = (ax1, ay1, ax2, ay2)

                # dibujar cuadricula (para visual)
                cell_w = (ax2 - ax1) // 3
                cell_h = (ay2 - ay1) // 3
                puntajes = [[5, 2, 5],
                            [3, 1, 3],
                            [5, 1, 5]]
                for i in range(3):
                    for j in range(3):
                        cx1 = ax1 + j * cell_w
                        cy1 = ay1 + i * cell_h
                        cx2 = cx1 + cell_w
                        cy2 = cy1 + cell_h
                        cv2.rectangle(original_frame, (cx1, cy1), (cx2, cy2), (0, 255, 0), 1)
                        cv2.putText(original_frame, str(puntajes[i][j]), (cx1 + 5, cy1 + 20),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            out.write(original_frame)

        cap.release()
        out.release()

        # 3) Al final, calcular puntaje si hubo balón y arco
        puntaje_asignado = None
        if last_ball_bbox and last_arco_bbox:
            bx, by, bw, bh = last_ball_bbox
            ball_cx = bx + bw // 2
            ball_cy = by + bh // 2

            ax1, ay1, ax2, ay2 = last_arco_bbox
            if ax1 <= ball_cx <= ax2 and ay1 <= ball_cy <= ay2:
                cell_w = (ax2 - ax1) // 3
                cell_h = (ay2 - ay1) // 3
                puntajes = [[5, 2, 5],
                            [3, 1, 3],
                            [5, 1, 5]]
                col = int(min(max((ball_cx - ax1) // cell_w, 0), 2))
                row = int(min(max((ball_cy - ay1) // cell_h, 0), 2))
                puntaje_asignado = puntajes[row][col]

        if puntaje_asignado is not None:
            print(f"Puntaje asignado: {puntaje_asignado}")
        else:
            print("No se detectó un gol en el video.")

        return output_path


In [ ]:
processor = VideoProcessor("/content/best.pt")
video_procesado = processor.process_video(
    input_path=r"/content/Sample3-recortado.mp4",
    output_path=r"/content/Sample3Gol.mp4"
)

Puntaje asignado: 5




---

